In [ ]:
import re
import glob
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D

In [ ]:
files = glob.glob('C:/Users/Dan_Yoo/ds-cs-project1/images/*')
_class = [x.split('/')[-1].replace('.jpg','') for x in files]

In [ ]:
X = []
for file_name in tqdm(files):
    img = image.load_img(file_name, target_size = (100, 100))
    v = image.img_to_array(img)
    v = np.expand_dims(v, axis=0)
    if len(X) > 0:
        X = np.concatenate((X, v))
    else:
        X = v

In [ ]:
p = re.compile("[^0-9]")
Y = ["".join(p.findall(x)) for x in _class]

In [ ]:
X = tf.keras.applications.mobilenet.preprocess_input(X)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(Y)
Y = to_categorical(y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2021)

In [ ]:
mobileNetModel = MobileNet(weights = 'imagenet', include_top = False)

model = Sequential()
model.add(mobileNetModel)
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation = 'softmax', kernel_initializer = 'he_normal'))
model.compile(optimizer = 'adam', 
                loss = 'categorical_crossentropy', 
                metrics = ['acc']
             )

In [ ]:
history = model.fit(X_train, Y_train, 
                    batch_size = 64, 
                    epochs = 10, 
                    verbose = 1, 
                    validation_split = 0.2)

In [ ]:
acc = model.evaluate(X_test, Y_test)
print('Loss: {:0.3f} | Accuracy: {:0.3f}'.format(acc[0],acc[1]))

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss) + 1)

plt.figure(figsize = (13,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, acc, 'r', label='accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Accuracy and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
plt.close()

In [ ]:
def pred(img_path):
    img = image.load_img(img_path, target_size = (100, 100))
    v = image.img_to_array(img)
    v = np.expand_dims(v, axis=0)
    X = tf.keras.applications.mobilenet.preprocess_input(v)
    pred = model.predict(X)
    prob = np.max(pred)
    r = np.argmax(pred)
    mylist = ['피자','치킨','스시','돈카츠','맥주']
    dct = {k:v for k, v in enumerate(mylist)}
    print ('pred:',dct[r])
    print ('prob: {}%'.format(prob * 100))

In [ ]:
pred('C:/Users/Dan_Yoo/ds-cs-project1/test/스시.jpg')